I will use GPT2 which is free.

In [1]:
import os

os.chdir('..')
os.listdir()

['.git',
 '.gitignore',
 '.idea',
 'data',
 'experiments',
 'fine_tuned',
 'notebooks',
 'out',
 'README.md',
 'shakespeare_data',
 'utils.py',
 '__pycache__']

In [2]:
from utils import get_model_tokenizer

model_raw_name = 'gpt2'
model, tokenizer = get_model_tokenizer(model_raw_name)

C:\Users\GIGABYTE\anaconda3\envs\tf_gpu\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.2) or chardet (None)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


In [3]:
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling

data = 'data/enhanced'
train_file = os.path.join(data, "train.txt")
test_file = os.path.join(data, "test.txt")


data_col = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
eval_dataset = TextDataset(tokenizer=tokenizer, file_path=test_file, block_size=1024)
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_file, block_size=1024)

C:\Users\GIGABYTE\anaconda3\envs\tf_gpu\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [4]:
output_dir = 'experiments'

train_args = TrainingArguments(
    save_steps=10_000,
    save_total_limit=2,
    num_train_epochs = 5,
    output_dir = output_dir,
    overwrite_output_dir=True,
    evaluation_strategy = 'epoch',
    per_device_train_batch_size = 8
)

trainer = Trainer(
    model = model,
    args = train_args,
    data_collator = data_col,
    eval_dataset = eval_dataset,
    train_dataset = train_dataset
)

In [12]:
from utils import launch_tensorboard

launch_tensorboard(log_dir='experiments', port=6666)

TensorBoard is running at http://localhost:6666/


In [6]:
print(model.device)
out = trainer.train()
if out:
    print(out)

C:\Users\GIGABYTE\anaconda3\envs\tf_gpu\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 298
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 190
  Number of trainable parameters = 124439808


cuda:0


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 31
  Batch size = 8
***** Running Evaluation *****
  Num examples = 31
  Batch size = 8
***** Running Evaluation *****
  Num examples = 31
  Batch size = 8
***** Running Evaluation *****
  Num examples = 31
  Batch size = 8
***** Running Evaluation *****
  Num examples = 31
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=190, training_loss=3.4696311549136514, metrics={'train_runtime': 350.4479, 'train_samples_per_second': 4.252, 'train_steps_per_second': 0.542, 'total_flos': 778650255360000.0, 'train_loss': 3.4696311549136514, 'epoch': 5.0})


In [7]:
trainer.save_model()
tokenizer.save_pretrained(output_dir)

Saving model checkpoint to experiments
Configuration saved in experiments\config.json
Configuration saved in experiments\generation_config.json
Model weights saved in experiments\pytorch_model.bin
tokenizer config file saved in experiments\tokenizer_config.json
Special tokens file saved in experiments\special_tokens_map.json


('experiments\\tokenizer_config.json',
 'experiments\\special_tokens_map.json',
 'experiments\\vocab.json',
 'experiments\\merges.txt',
 'experiments\\added_tokens.json')

## Inference

In [2]:
from utils import get_model_tokenizer

inference_model_name = "../experiments"
inference_model, inference_tokenizer = get_model_tokenizer(inference_model_name)

In [4]:
from utils import  generate_text

input_text = "Love or Hate, that is the question:"
generated_text = generate_text(inference_model, inference_tokenizer, input_text, max_length=200)
print(generated_text)

Love or Hate, that is the question:
And if you have not, you are not a man
To be hated.

KING RICHARD II: I do not know, sir, but I know
That you do hate me. I have been a prisoner
Of your tyranny, and I am a traitor to you. You
Have made me a slave to your power,
For I did not love you, nor did I love
Your majesty. But I will not be a king, for
I am not your king. Therefore, I'll be
A king to the people. If you will, then I shall be king
In the land of England. For I hate you
More than you hate yourself. What, my lord, is this?
What, what, this is a lie? What is it? I
Am a prince, a nobleman, an honest man,--
The king of France, the king and queen of



In [11]:
from utils import  generate_text

input_text = "My country is under attack, fight or die, believe or hope?"
generated_text = generate_text(inference_model, inference_tokenizer, input_text, max_length=50)
print(generated_text)

My country is under attack, fight or die, believe or hope?

KING RICHARD II:
I have heard the news of the king's death.
The king is dead, and the queen is in the womb. I

